<a href="https://colab.research.google.com/github/igormatheusdiniz/ri_lab_05/blob/master/ri_lab5_igor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1797]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import math
import bisect
import nltk
nltk.download('rslp')
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize
from collections import Counter
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize


[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
url = 'https://raw.githubusercontent.com/Benardi/ri_lab_01/master/output/results.csv'

Utilizei a base de dados de Benardi, e por motivos de não precisar importar o .csv toda vez ao inicializar o colab, defini a url em meu lab.

# Tratamento dos dados

In [0]:
dataframe = pd.read_csv(url)
dataframe['treatedText'] = dataframe.text.apply(lambda x: re.sub('[^0-9A-ZÀ-Úa-zà-ú\']', ' ', x.lower()))

Diferente do lab anterior onde para obter os tokens, não utilizei o método tokenize do ntlk, para atender as especificações do lab atual, atualizei a forma que recupero os tokens do meu texto.

In [0]:
regex = RegexpTokenizer(r'[A-zÀ-ú-]+')
stopWords = set(stopwords.words('portuguese'))

# Algoritimos necessários para este lab

## Algoritimo de construção de índice Invertido

In [0]:


def invertedIndex(documents):
  I = {}
  index = 0
  
  ##Para cada documento na minha relação de documentos irei realizar o processo de tokenização
  for document in documents.treatedText:
    index += 1
    T = [token for token in regex.tokenize(document)
        if((len(token)>=3) and token not in stopWords and token[0] != "[" and token[-1]!="]")]
    
    ##Por motivos didáticos, resolvi ordenar minha lista de tokens
    T.sort()
    
    ## Pra cada token criado se ele ainda não existe na minha estrutura de armazenamento como uma chave, eu crio e em seguida adiciono
    ##o indice do documento como valor para esta chave
    ## Caso já exista, apenas adiciona o índice do documento para esta chave
    for token in T:
      if token not in I.keys():
        I[token]=[]
       
      pair = []
      ocurrence = T.count(token)
     
      pair.append(index)
      pair.append(ocurrence)
      if(pair not in I[token]):
        I[token].append(pair)
  ## Por motivos estéticos resolvi comentar esta linha para não ficar uma lista de print muito grande no gitHub, dificultando a visualização do documento como um todo      
  ##print(I)
  return I

invertedIndex = invertedIndex(dataframe)

## Algoritimo Termo por vez

In [1802]:
def termAtTimeRetrieval(terms, invertedIndex, k):
  hashtable = {}
  invertedLists =[]
  R = []
  
  ##Para cada termo recebido, se este termo está contindo nas chaves do meu IndiceInvertido construído anteriormente
  ##adicione na minha estrutura de listas invertidas 
  for term in terms.split():
    if term in invertedIndex.keys():
      invertedLists.append(invertedIndex[term])
  
  
  ## DIferente do algoritimo documentoPorVez iremos ACUMULAR os scores para cada termo
  ## Para isso analisamos, os valores de cada chave/termo em nossa estrutura
  ## nosso value 0 será o "índice" do documento, enquanto que o value 1 a frequência
  ## Se nosso documento ja existir na hashtable de retorno iremos apenas atualizar o score acumulando-o
  ## Caso contrário iremos cria-ló em nossa hashtable e atribuir um score = frequencia para ele.
  
  for invertedList in invertedLists:
    for value in invertedList:
      d = value[0]
      f = value[1]
      if d in hashtable.keys():
        hashtable[d] = hashtable[d] + f
      else:  
        hashtable[d] = f
  
  R = list(map(lambda elem: (elem[1],elem[0]), hashtable.items()))
  
  ##Ordenando nossa estrutura de resposta, e selecionando os k-ésimos elementos
  R.sort(reverse=True) 
  result = R[0:k]
  
  return result

resultado = documentAtTimeRetrieval('juíza federal', invertedIndex, 5, dataframe)

print(resultado)

[(5, 151), (4, 173), (4, 1), (3, 248), (3, 229)]


In [0]:
## Definindo 5 consultas de um termo, para testarmos ambos algoritimos
consultas = ['escola','bolsonaro', 'lula', 'universidades', 'infantil']

# Questão 1

In [0]:
documents = dataframe.treatedText.count()

In [1805]:
def dices_coef(term, invertedIndex):
  rank = []
  result = []
  score = 0
  na = len(set(map(lambda x: x[0], invertedIndex[term])))
  nb = 0
  nab = 0
  for index in invertedIndex:
    if(index != term):
      set_aux = set(map(lambda x: x[0], invertedIndex[term])) & set(map(lambda x: x[0], invertedIndex[index]))
      nab = len(set_aux)
   
      set_aux2 = set(map(lambda x: x[0], invertedIndex[index]))
      nb = len(set_aux2)
    
      score = (nab/(na+nb))
                   
      rank.append((score, index))
    
  rank.sort(reverse=True) 
  for i in range(10):
    result.append(rank[i][1])
  #result = rank[0:10]
  
  return result

teste = dices_coef("jair", invertedIndex)
teste

  

['bolsonaro',
 'existia',
 'vinha',
 'inclusive',
 'acabaram',
 'precisam',
 'explícito',
 'ordem',
 'militar',
 'tensão']

In [1806]:
def mutual_info(term, invertedIndex):
  rank = []
  result =[]
  score = 0
  na = len(set(map(lambda x: x[0], invertedIndex[term])))
  nb = 0
  nab = 0
  for index in invertedIndex:
    if(index != term):
      set_aux = set(map(lambda x: x[0], invertedIndex[term])) & set(map(lambda x: x[0], invertedIndex[index]))
      nab = len(set_aux)
   
      set_aux2 = set(map(lambda x: x[0], invertedIndex[index]))
      nb = len(set_aux2)
    
      score = (nab/(na*nb))
                   
      rank.append((score, index))
    
  rank.sort(reverse=True) 
  for i in range(10):
    result.append(rank[i][1])
  #result = rank[0:10]
    
  return result

teste = mutual_info("jair", invertedIndex)
teste

['óbvias',
 'ético',
 'ângulos',
 'ângelo',
 'àquelas',
 'zelenskiy',
 'yuriy',
 'yulia',
 'yanukovich',
 'xenófobos']

In [1807]:
def chi_square(term, invertedIndex, documents):
  rank = []
  result = []
  score = 0
  na = len(set(map(lambda x: x[0], invertedIndex[term])))
  nb = 0
  nab = 0
  for index in invertedIndex:
    if(index != term):
      set_aux = set(map(lambda x: x[0], invertedIndex[term])) & set(map(lambda x: x[0], invertedIndex[index]))
      nab = len(set_aux)
   
      set_aux2 = set(map(lambda x: x[0], invertedIndex[index]))
      nb = len(set_aux2)
    
      score = ((nab - (1/documents) * na * nb)**2 /(na*nb))
                   
      rank.append((score, index))
    
  rank.sort(reverse=True) 
  for i in range(10):
    result.append(rank[i][1])
  #result = rank[0:10]
    
  return result

teste = chi_square("jair", invertedIndex, documents)
teste

['bolsonaro',
 'explícito',
 'identificou',
 'gestor',
 'existia',
 'gomes',
 'ustra',
 'torturados',
 'raízes',
 'queiroz']

In [1808]:
def expected_Mutual_Information(term, invertedIndex, documents):
  rank = []
  result = []
  score = 0
  na = len(set(map(lambda x: x[0], invertedIndex[term])))
  nb = 0
  nab = 0
  for index in invertedIndex:
    if(index != term):
      set_aux = set(map(lambda x: x[0], invertedIndex[term])) & set(map(lambda x: x[0], invertedIndex[index]))
      nab = len(set_aux)
   
      set_aux2 = set(map(lambda x: x[0], invertedIndex[index]))
      nb = len(set_aux2)
      
      if(nab != 0 and na != 0 and nb != 0 and documents!=0):
        score = nab * math.log(documents*(nab/(na*nb)))
                   
      rank.append((score, index))
    
  rank.sort(reverse=True)
  for i in range(10):
    result.append(rank[i][1])
  #result = rank[0:10]
    
  return result

teste = expected_Mutual_Information("jair", invertedIndex, documents)
teste
  

['bolsonaro',
 'indígena',
 'incômodo',
 'inclusive',
 'militar',
 'pública',
 'brasileira',
 'brasileiro',
 'existia',
 'militares']

##Criando as tabelas pra cada consulta


In [0]:
#consultas = ['escola','bolsonaro', 'lula', 'universidades', 'infantil']
documents = dataframe.treatedText.count() 

### Palavra: escola

In [1810]:
MIM1 = []
EMIM1 = []
X21 = []
DICE1 = []


table1 = pd.DataFrame()
pd.options.display.max_colwidth = 160
  
MIM1 = mutual_info("escola",invertedIndex)
EMIM1 = expected_Mutual_Information("escola", invertedIndex, documents)
X21 = chi_square("escola", invertedIndex, documents)
DICE1 = dices_coef("escola", invertedIndex)

table1['MIM'] = MIM1
table1['EMIM'] = EMIM1
table1['X2'] = X21
table1['DICE'] = DICE1
  
table1
  
  

,MIM,EMIM,X2,DICE
0,óbvias,universidade,professores,querem
1,íntegra,jovens,escolar,universidade
2,índios,querem,converter,jovens
3,étnicos,professor,araguaia,professor
4,étnico,criar,querem,criar
5,ético,enriquecer,superior,educação
6,épocas,embruteceu,professor,jovem
7,árbitros,educação,quadros,discurso
8,zuenir,professores,massacre,qualidade
9,zombava,cada,universidade,superior


### Palavra: bolsonaro

In [1811]:
MIM2 = []
EMIM2 = []
X22 = []
DICE2 = []


table2 = pd.DataFrame()
pd.options.display.max_colwidth = 160
  
MIM2 = mutual_info("bolsonaro",invertedIndex)
EMIM2 = expected_Mutual_Information("bolsonaro", invertedIndex, documents)
X22 = chi_square("bolsonaro", invertedIndex, documents)
DICE2 = dices_coef("bolsonaro", invertedIndex)

table2['MIM'] = MIM2
table2['EMIM'] = EMIM2
table2['X2'] = X22
table2['DICE'] = DICE2
  
table2
  

,MIM,EMIM,X2,DICE
0,ótimas,presidente,jair,presidente
1,óbvios,brasil,psl,brasil
2,óbvias,governo,previdência,governo
3,índole,brasileiro,reforma,brasileiro
4,ético,ministro,planalto,ministro
5,ânus,reforma,brasileiro,político
6,ânimos,político,brasil,brasileira
7,ângulos,psl,deputado,ainda
8,ângelo,jair,presidente,paulo
9,ávidos,prima,guedes,federal


### Palavra: lula

In [1812]:
MIM3 = []
EMIM3 = []
X23 = []
DICE3 = []


table3 = pd.DataFrame()
pd.options.display.max_colwidth = 160
  
MIM3 = mutual_info("lula",invertedIndex)
EMIM3 = expected_Mutual_Information("lula", invertedIndex, documents)
X23 = chi_square("lula", invertedIndex, documents)
DICE3 = dices_coef("lula", invertedIndex)

table3['MIM'] = MIM3
table3['EMIM'] = EMIM3
table3['X2'] = X23
table3['DICE'] = DICE3
  
table3
  

,MIM,EMIM,X2,DICE
0,ácidas,inácio,inácio,inácio
1,xavier,bolsonaro,péssimo,silva
2,vomitando,póstuma,psicanálise,michel
3,voltas,província,pareceram,entregar
4,vitoriosos,provas,minimiza,prisão
5,vitimizava,programação,interrogado,recado
6,vermelhas,programada,governabilidade,temer
7,vavá,prisão,eleitorado,psol
8,vaso,suspender,desidratação,negativa
9,vacina,supremo,derrotados,lava


### Palavra: universidades

In [1813]:
MIM4 = []
EMIM4 = []
X24 = []
DICE4 = []


table4 = pd.DataFrame()
pd.options.display.max_colwidth = 160
  
MIM4 = mutual_info("universidades",invertedIndex)
EMIM4 = expected_Mutual_Information("universidades", invertedIndex, documents)
X24 = chi_square("universidades", invertedIndex, documents)
DICE4 = dices_coef("universidades", invertedIndex)

table4['MIM'] = MIM4
table4['EMIM'] = EMIM4
table4['X2'] = X24
table4['DICE'] = DICE4
  
table4

,MIM,EMIM,X2,DICE
0,úlceras,precisa,concentrar,desastre
1,étnicos,desastre,harvard,harvard
2,árvore,problema,aulas,aulas
3,árbitros,harvard,desastre,concentrar
4,yfn,aulas,yale,estudantes
5,xingar,muitos,robert,ocorre
6,wynn,olavo,vividos,yale
7,witold,ocorre,universitária,robert
8,wilstermann,originalmente,tremendo,financiamento
9,what,olhava,tratada,precisa


### Palavra: Infantil

In [1814]:
MIM5 = []
EMIM5 = []
X25 = []
DICE5 = []


table5 = pd.DataFrame()
pd.options.display.max_colwidth = 160
  
MIM5 = mutual_info("infantil",invertedIndex)
EMIM5 = expected_Mutual_Information("infantil", invertedIndex, documents)
X25 = chi_square("infantil", invertedIndex, documents)
DICE5 = dices_coef("infantil", invertedIndex)

table5['MIM'] = MIM5
table5['EMIM'] = EMIM5
table5['X2'] = X25
table5['DICE'] = DICE5
  
table5

,MIM,EMIM,X2,DICE
0,éticas,sexual,keaton,sexual
1,zumbido,crime,eva,keaton
2,youtuber,crianças,danny,eva
3,whisky,michael,burton,danny
4,wee,keaton,tim,burton
5,warner,eva,elefante,tim
6,walt,danny,sexual,elefante
7,wahlberg,burton,seminário,michael
8,válida,perguntaram,semanais,james
9,voltarmos,perguntar,remake,infância


## Qual métrica você acha que obteve os melhores resultados? Por que?

Os melhores resultados foram do EMIM e o DICE, levando em consideração que seus resultados são mais generalistas e tem mais relação com o termo pesquisado. É possível perceber que a abordagem MIM, é bem ruim, termos sem relação com a palavra aparecem no top 10, isso occore porque ele  prioriza as palavras que aparecem menos, julgando-as como mais importantes fazendo com que os termos retornados com essa métrica são mais específicos

# Questão 2

In [0]:
#consultas = ['escola','bolsonaro', 'lula', 'universidades', 'infantil']

In [0]:
def top_docs(query, k):
  
  top_docs = [doc for score,doc in termAtTimeRetrieval(query, invertedIndex, k)]
  top_docs_ranking = {} 
  
  for word in invertedIndex:
    top_docs_ranking[word] = []
    for doc_score in invertedIndex[word]:
      if doc_score[0] in top_docs:
        top_docs_ranking[word].append(doc_score)
        
  return top_docs_ranking

In [0]:
def conj_and(terms, invertedIndex, k):
  indexes = []
  rank = []
  
  ##Para cada termo recebido, se este termo está contindo nas chaves do meu IndiceInvertido construído anteriormente adicione na minha estrutura de listas invertidas 
  for term in terms.split():
    if term in invertedIndex.keys():
        indexes.append(invertedIndex[term])
  
  termos = [item for sublist in indexes for item in sublist]
  termos.sort()
  for i in range(len(termos)):
    score = 0
    d = termos.pop()
    aux = 1
    for index in termos:
      if index[0] == d[0]:
        score += index[1] + d[1]
        aux += 1
    if score != 0 and aux == len(indexes):
      hp.heappush(rank, (d[0], score))
  

  return hp.nlargest(k, rank)

#conj_and('juíza federal', invertedIndex, 10)

In [0]:
consultas = []
resultados = []


## Palavra: escola

In [0]:
consulta = expected_Mutual_Information("escola", invertedIndex, documents)
consulta10 = ' '.join(map(str, consulta))
consultas.append(consulta10)
resultado = conj_and(consulta10, invertedIndex, 10)
resultados.append(resultado)

In [0]:
consulta5 = consulta[0:5]
consulta5 = ' '.join(map(str, consulta5))
consultas.append(consulta5)
resultado = conj_and(consulta5, invertedIndex, 10)
resultados.append(resultado)

In [0]:
consulta3 = consulta[0:3]
consulta3 = ' '.join(map(str, consulta3))
consultas.append(consulta3)
resultado = conj_and(consulta3, invertedIndex, 10)
resultados.append(resultado)

In [1822]:
table_question2 = pd.DataFrame()
pd.options.display.max_colwidth = 160

table_question2['Consultas'] = consultas
table_question2['Documentos'] = resultados
table_question2

,Consultas,Documentos
0,universidade jovens querem professor criar enriquecer embruteceu educação professores cada,[]
1,universidade jovens querem professor criar,[]
2,universidade jovens querem,"[(214, 12)]"


## Palavra: Bolsonaro

In [0]:
consulta = expected_Mutual_Information("bolsonaro", invertedIndex, documents)
consulta10 = ' '.join(map(str, consulta))
consultas.append(consulta10)
resultado = conj_and(consulta10, invertedIndex, 10)
resultados.append(resultado)

In [0]:
consulta5 = consulta[0:5]
consulta5 = ' '.join(map(str, consulta5))
consultas.append(consulta5)
resultado = conj_and(consulta5, invertedIndex, 10)
resultados.append(resultado)

In [0]:
consulta3 = consulta[0:3]
consulta3 = ' '.join(map(str, consulta3))
consultas.append(consulta3)
resultado = conj_and(consulta3, invertedIndex, 10)
resultados.append(resultado)

In [1826]:
table_question2 = pd.DataFrame()
pd.options.display.max_colwidth = 160

table_question2['Consultas'] = consultas
table_question2['Documentos'] = resultados
table_question2

,Consultas,Documentos
0,universidade jovens querem professor criar enriquecer embruteceu educação professores cada,[]
1,universidade jovens querem professor criar,[]
2,universidade jovens querem,"[(214, 12)]"
3,presidente brasil governo brasileiro ministro reforma político psl jair prima,[]
4,presidente brasil governo brasileiro ministro,"[(218, 21), (215, 44), (174, 36), (173, 74), (166, 98), (151, 205), (115, 57), (105, 36), (95, 30), (42, 36)]"
5,presidente brasil governo,"[(239, 4), (238, 21), (235, 23), (232, 7), (229, 26), (228, 13), (223, 11), (222, 9), (218, 11), (215, 15)]"


## Palavra: lula

In [0]:
consulta = expected_Mutual_Information("lula", invertedIndex, documents)
consulta10 = ' '.join(map(str, consulta))
consultas.append(consulta10)
resultado = conj_and(consulta10, invertedIndex, 10)
resultados.append(resultado)

In [0]:
consulta5 = consulta[0:5]
consulta5 = ' '.join(map(str, consulta5))
consultas.append(consulta5)
resultado = conj_and(consulta5, invertedIndex, 10)
resultados.append(resultado)

In [0]:
consulta3 = consulta[0:3]
consulta3 = ' '.join(map(str, consulta3))
consultas.append(consulta3)
resultado = conj_and(consulta3, invertedIndex, 10)
resultados.append(resultado)

In [1830]:
table_question2 = pd.DataFrame()
pd.options.display.max_colwidth = 160

table_question2['Consultas'] = consultas
table_question2['Documentos'] = resultados
table_question2

,Consultas,Documentos
0,universidade jovens querem professor criar enriquecer embruteceu educação professores cada,[]
1,universidade jovens querem professor criar,[]
2,universidade jovens querem,"[(214, 12)]"
3,presidente brasil governo brasileiro ministro reforma político psl jair prima,[]
4,presidente brasil governo brasileiro ministro,"[(218, 21), (215, 44), (174, 36), (173, 74), (166, 98), (151, 205), (115, 57), (105, 36), (95, 30), (42, 36)]"
5,presidente brasil governo,"[(239, 4), (238, 21), (235, 23), (232, 7), (229, 26), (228, 13), (223, 11), (222, 9), (218, 11), (215, 15)]"
6,inácio bolsonaro póstuma província provas programação programada prisão suspender supremo,[]
7,inácio bolsonaro póstuma província provas,[]
8,inácio bolsonaro póstuma,[]


## Palavra: universidades

In [0]:
consulta = expected_Mutual_Information("universidades", invertedIndex, documents)
consulta10 = ' '.join(map(str, consulta))
consultas.append(consulta10)
resultado = conj_and(consulta10, invertedIndex, 10)
resultados.append(resultado)

In [0]:
consulta5 = consulta[0:5]
consulta5 = ' '.join(map(str, consulta5))
consultas.append(consulta5)
resultado = conj_and(consulta5, invertedIndex, 10)
resultados.append(resultado)

In [0]:
consulta3 = consulta[0:3]
consulta3 = ' '.join(map(str, consulta3))
consultas.append(consulta3)
resultado = conj_and(consulta3, invertedIndex, 10)
resultados.append(resultado)

In [1834]:
table_question2 = pd.DataFrame()
pd.options.display.max_colwidth = 160

table_question2['Consultas'] = consultas
table_question2['Documentos'] = resultados
table_question2

,Consultas,Documentos
0,universidade jovens querem professor criar enriquecer embruteceu educação professores cada,[]
1,universidade jovens querem professor criar,[]
2,universidade jovens querem,"[(214, 12)]"
3,presidente brasil governo brasileiro ministro reforma político psl jair prima,[]
4,presidente brasil governo brasileiro ministro,"[(218, 21), (215, 44), (174, 36), (173, 74), (166, 98), (151, 205), (115, 57), (105, 36), (95, 30), (42, 36)]"
5,presidente brasil governo,"[(239, 4), (238, 21), (235, 23), (232, 7), (229, 26), (228, 13), (223, 11), (222, 9), (218, 11), (215, 15)]"
6,inácio bolsonaro póstuma província provas programação programada prisão suspender supremo,[]
7,inácio bolsonaro póstuma província provas,[]
8,inácio bolsonaro póstuma,[]
9,precisa desastre problema harvard aulas muitos olavo ocorre originalmente olhava,[]


## Palavra: infantil

In [0]:
consulta = expected_Mutual_Information("infantil", invertedIndex, documents)
consulta10 = ' '.join(map(str, consulta))
consultas.append(consulta10)
resultado = conj_and(consulta10, invertedIndex, 10)
resultados.append(resultado)

In [0]:
consulta5 = consulta[0:5]
consulta5 = ' '.join(map(str, consulta5))
consultas.append(consulta5)
resultado = conj_and(consulta5, invertedIndex, 10)
resultados.append(resultado)

In [0]:
consulta3 = consulta[0:3]
consulta3 = ' '.join(map(str, consulta3))
consultas.append(consulta3)
resultado = conj_and(consulta3, invertedIndex, 10)
resultados.append(resultado)

In [1838]:
table_question2 = pd.DataFrame()
pd.options.display.max_colwidth = 160

table_question2['Consultas'] = consultas
table_question2['Documentos'] = resultados
table_question2

,Consultas,Documentos
0,universidade jovens querem professor criar enriquecer embruteceu educação professores cada,[]
1,universidade jovens querem professor criar,[]
2,universidade jovens querem,"[(214, 12)]"
3,presidente brasil governo brasileiro ministro reforma político psl jair prima,[]
4,presidente brasil governo brasileiro ministro,"[(218, 21), (215, 44), (174, 36), (173, 74), (166, 98), (151, 205), (115, 57), (105, 36), (95, 30), (42, 36)]"
5,presidente brasil governo,"[(239, 4), (238, 21), (235, 23), (232, 7), (229, 26), (228, 13), (223, 11), (222, 9), (218, 11), (215, 15)]"
6,inácio bolsonaro póstuma província provas programação programada prisão suspender supremo,[]
7,inácio bolsonaro póstuma província provas,[]
8,inácio bolsonaro póstuma,[]
9,precisa desastre problema harvard aulas muitos olavo ocorre originalmente olhava,[]


## TOP 3

In [0]:
k = 3
ranking_docs1 = top_docs("escola", k)
ranking_docs2 = top_docs("bolsonaro", k)
ranking_docs3 = top_docs("lula", k)
ranking_docs4 = top_docs("universidades", k)
ranking_docs5 = top_docs("infantil", k)

In [0]:
table_top1 = dices_coef("escola", ranking_docs1)
table_top2 = dices_coef("bolsonaro", ranking_docs2)
table_top3 = dices_coef("lula", ranking_docs3)
table_top4 = dices_coef("universidades", ranking_docs4)
table_top5 = dices_coef("infantil", ranking_docs5)

In [1841]:
table_emim = pd.DataFrame()

table_emim['Dice Antes'] = table1['DICE']
table_emim['Dice Depois'] = table_top1

table_emim

,Dice Antes,Dice Depois
0,querem,última
1,universidade,ter
2,jovens,sido
3,professor,ser
4,criar,professores
5,educação,presidente
6,jovem,pessoa
7,discurso,parte
8,qualidade,massacre
9,superior,lidar


In [1842]:
table_emim2 = pd.DataFrame()

table_emim2['Dice Antes'] = table2['DICE']
table_emim2['Dice Depois'] = table_top2

table_emim2

,Dice Antes,Dice Depois
0,presidente,vez
1,brasil,três
2,governo,trata
3,brasileiro,trabalho
4,ministro,todas
5,político,sobre
6,brasileira,sob
7,ainda,site
8,paulo,sendo
9,federal,república


In [1843]:
table_emim3 = pd.DataFrame()

table_emim3['Dice Antes'] = table3['DICE']
table_emim3['Dice Depois'] = table_top3

table_emim3

,Dice Antes,Dice Depois
0,inácio,ter
1,silva,ser
2,michel,presidente
3,entregar,político
4,prisão,pode
5,recado,outros
6,temer,outro
7,psol,contra
8,negativa,brasil
9,lava,bolsonaro


In [1844]:
table_emim4 = pd.DataFrame()

table_emim4['Dice Antes'] = table4['DICE']
table_emim4['Dice Depois'] = table_top4

table_emim4

,Dice Antes,Dice Depois
0,desastre,universidade
1,harvard,todos
2,aulas,ter
3,concentrar,sobre
4,estudantes,ser
5,ocorre,quatro
6,yale,quase
7,robert,presidente
8,financiamento,pessoas
9,precisa,mundo


In [1845]:
table_emim5 = pd.DataFrame()

table_emim5['Dice Antes'] = table5['DICE']
table_emim5['Dice Depois'] = table_top5

table_emim5

,Dice Antes,Dice Depois
0,sexual,tempo
1,keaton,sobre
2,eva,ser
3,danny,primeira
4,burton,precisa
5,tim,parte
6,elefante,meio
7,michael,hoje
8,james,filho
9,infância,fato


## TOP 5

In [0]:
k = 5
ranking_docs1 = top_docs("escola", k)
ranking_docs2 = top_docs("bolsonaro", k)
ranking_docs3 = top_docs("lula", k)
ranking_docs4 = top_docs("universidades", k)
ranking_docs5 = top_docs("infantil", k)

In [0]:
table_top1 = dices_coef("escola", ranking_docs1)
table_top2 = dices_coef("bolsonaro", ranking_docs2)
table_top3 = dices_coef("lula", ranking_docs3)
table_top4 = dices_coef("universidades", ranking_docs4)
table_top5 = dices_coef("infantil", ranking_docs5)

In [1848]:
table_dice = pd.DataFrame()

table_dice['Dice Antes'] = table1['DICE']
table_dice['Dice Depois'] = table_top1

table_dice

,Dice Antes,Dice Depois
0,querem,ter
1,universidade,sido
2,jovens,ser
3,professor,pessoa
4,criar,parte
5,educação,duas
6,jovem,dias
7,discurso,anos
8,qualidade,última
9,superior,violência


In [1849]:
table_dice2 = pd.DataFrame()

table_dice2['Dice Antes'] = table2['DICE']
table_dice2['Dice Depois'] = table_top2

table_dice2

,Dice Antes,Dice Depois
0,presidente,sobre
1,brasil,quase
2,governo,presidente
3,brasileiro,porém
4,ministro,pessoas
5,político,outro
6,brasileira,militar
7,ainda,fazer
8,paulo,envolvendo
9,federal,conta


In [1850]:
table_dice3 = pd.DataFrame()

table_dice3['Dice Antes'] = table3['DICE']
table_dice3['Dice Depois'] = table_top3

table_dice3

,Dice Antes,Dice Depois
0,inácio,ser
1,silva,presidente
2,michel,todos
3,entregar,ter
4,prisão,sobre
5,recado,prisão
6,temer,preso
7,psol,político
8,negativa,pode
9,lava,outros


In [1851]:
table_dice4 = pd.DataFrame()

table_dice4['Dice Antes'] = table4['DICE']
table_dice4['Dice Depois'] = table_top4

table_dice4

,Dice Antes,Dice Depois
0,desastre,ter
1,harvard,ser
2,aulas,presidente
3,concentrar,diz
4,estudantes,anos
5,ocorre,universidade
6,yale,trabalhar
7,robert,sobre
8,financiamento,quase
9,precisa,precisa


In [1852]:
table_dice5 = pd.DataFrame()

table_dice5['Dice Antes'] = table5['DICE']
table_dice5['Dice Depois'] = table_top5

table_dice5

,Dice Antes,Dice Depois
0,sexual,anos
1,keaton,ainda
2,eva,tempo
3,danny,sobre
4,burton,ser
5,tim,sempre
6,elefante,público
7,michael,precisa
8,james,muitos
9,infância,infância


## TOP 10

In [0]:
k = 10
ranking_docs1 = top_docs("escola", k)
ranking_docs2 = top_docs("bolsonaro", k)
ranking_docs3 = top_docs("lula", k)
ranking_docs4 = top_docs("universidades", k)
ranking_docs5 = top_docs("infantil", k)

In [0]:
table_top1 = dices_coef("escola", ranking_docs1)
table_top2 = dices_coef("bolsonaro", ranking_docs2)
table_top3 = dices_coef("lula", ranking_docs3)
table_top4 = dices_coef("universidades", ranking_docs4)
table_top5 = dices_coef("infantil", ranking_docs5)

In [1855]:
table_dice = pd.DataFrame()

table_dice['Dice Antes'] = table1['DICE']
table_dice['Dice Depois'] = table_top1

table_dice

,Dice Antes,Dice Depois
0,querem,sobre
1,universidade,ser
2,jovens,anos
3,professor,ter
4,criar,segundo
5,educação,pessoas
6,jovem,onde
7,discurso,maior
8,qualidade,tudo
9,superior,presidente


In [1856]:
table_dice2 = pd.DataFrame()

table_dice2['Dice Antes'] = table2['DICE']
table_dice2['Dice Depois'] = table_top2

table_dice2

,Dice Antes,Dice Depois
0,presidente,presidente
1,brasil,fazer
2,governo,sobre
3,brasileiro,ser
4,ministro,dois
5,político,país
6,brasileira,outros
7,ainda,maior
8,paulo,disse
9,federal,dia


In [1857]:
table_dice3 = pd.DataFrame()

table_dice3['Dice Antes'] = table3['DICE']
table_dice3['Dice Depois'] = table_top3

table_dice3

,Dice Antes,Dice Depois
0,inácio,ser
1,silva,presidente
2,michel,bolsonaro
3,entregar,pode
4,prisão,outros
5,recado,ainda
6,temer,todos
7,psol,ter
8,negativa,político
9,lava,política


In [1858]:
table_dice4 = pd.DataFrame()

table_dice4['Dice Antes'] = table4['DICE']
table_dice4['Dice Depois'] = table_top4

table_dice4

,Dice Antes,Dice Depois
0,desastre,ter
1,harvard,ser
2,aulas,sobre
3,concentrar,outro
4,estudantes,mundo
5,ocorre,muitos
6,yale,desde
7,robert,brasil
8,financiamento,vez
9,precisa,presidente


In [1859]:
table_dice5 = pd.DataFrame()

table_dice5['Dice Antes'] = table5['DICE']
table_dice5['Dice Depois'] = table_top5

table_dice5

,Dice Antes,Dice Depois
0,sexual,anos
1,keaton,ainda
2,eva,ser
3,danny,crianças
4,burton,tempo
5,tim,sobre
6,elefante,segundo
7,michael,relação
8,james,parte
9,infância,forma


## Discussão

É perceptível que no top-3 algumas consultas acabam perdendo o sentido exibindo palavras que não possuem um significado forte para o contexto, diminuindo assim a qualidade dos resultados. Já para os top-5 e top-10 os resultados,são mais condizentes com o contexto uma vez que utilizam de  documentos que realmente descrevem o contexto da palavra consultada.